In [1]:
import sys
sys.path.append('/usr/local/lib/python3.5/site-packages')
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import os

In [2]:
carpeta = (os.getcwd()+'/tc02Data/')

# Nombres de los archivos: yeast_(...).txt
archivos = ['Y2H','AP-MS','LIT','LIT_Reguly']

# Lista donde se van a ir agregando los grafos en el orden de los archivos
Gs = []

for j,archivo in enumerate(archivos):
    data = pd.read_csv(carpeta+'yeast_'+archivo+'.txt', sep='\t', header=None)
    
    G = nx.Graph()
    
    for i in range(len(data)):
        G.add_edges_from([(data[0][i],data[1][i])])
    Gs.append(G)

# El último archivo, LIT_Reguly, es el único que tiene encabezado
# Quise poner header automático pero devuelve un error, así que elimino lo que sobra a lo bruto
Gs[3].remove_node("Bait gene/protein")
Gs[3].remove_node("Hit gene/protein")

In [3]:
# Proteinas esenciales
data_ess = pd.read_csv(carpeta+'Essential_ORFs_paperHe.txt', sep='\t', header=0,skipfooter=4,usecols=[1])

# Para eliminar los espacios en los nombres de las proteinas
data_ess['ORF_name'] = data_ess['ORF_name'].map(lambda x: x.strip())

ess = data_ess["ORF_name"].tolist()
del ess[0] # como antes, elimino el encabezado

# ess es la lista de proteinas esenciales

C:\Users\Luli\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  


In [4]:
# Agrego la caracteristica de ser o no esencial
from funciones import esenciales

for G in Gs:
    esenciales(G,ess)

In [ ]:
G = Gs[0]

Enlaces_iguales = [] # Lista donde se almacenan la cantidad de enlaces entre proteínas del mismo tipo

N_iter = 2000

nodos_lista = list(G.nodes())
ess_dict = nx.get_node_attributes(G,'essential')
ess_lista = list(ess_dict)


for it in range(N_iter):
    ess_shuffle = ess_lista
    np.random.shuffle(ess_shuffle) # Reordenamos aleat los generos para el resto de las iteraciones

    # Reasignamos a cada nodo un valor de esencialidad del vector de ess_shuffle:
    for nodo,esen in zip(nodos_lista,ess_shuffle):
        G.add_node(nodo, essential=esen)
        
    #Cuenta la fraccion de enlaces que conecta nodos de igual tipo:
    enlaces = list(G.edges.data())
    enlaces_igual = 0 # Ponemos en cero el contador de enlaces iguales

    # Recorremos los enlaces y nos fijamos cual de ellos es entre iguales:
    for i in range(0,len(enlaces)-1):
        esencialidad1 = G.nodes[enlaces[i][0]]['essential']
        esencialidad2 = G.nodes[enlaces[i][1]]['essential']
        if esencialidad1 == esencialidad2:   # Comparamos los tipos
            enlaces_igual += 1 # Incrementamos el contador si los tipos son iguales
    Enlaces_iguales.append(enlaces_igual) # Guardamos la cantidad de enlaces del mismo tipo en cada iteración

# Ahora tenemos una lista Enlaces_iguales con la cantidad de enlaces iguales para N iteraciones

# Valor medio y desviacion standar
mean_enlaces = np.mean(Enlaces_iguales)
desv_enlaces = np.std(Enlaces_iguales)

print ('Distribucion de enlaces del mismo tipo:')
print ('Valor medio(H null): {0:.2f}'.format(mean_enlaces))
print ('Desviacion Standar: {0:.2f}'.format(desv_enlaces))
#print ('Valor Red Real: {0:.2f}'.format(Enlaces_iguales[0])) #CAMBIAR ESTO

In [11]:
L = len(Gs)
Q = np.zeros(L)

for i, G in enumerate(Gs):
    N = G.number_of_nodes()
    m = G.number_of_edges()
    proteinas = list(G.nodes())
    grado = dict(G.degree())
    A = nx.to_numpy_matrix(G)

    # Matriz de variable categorica esencialidad: Cij=delta(ci,cj)
    C = np.zeros(np.shape(A))

    for ni, inodo in enumerate(proteinas):
        for nj, jnodo in enumerate(proteinas):
            if G.nodes[inodo]['essential']==G.nodes[jnodo]['essential']:
                C[ni,nj] = 1

    #Calculo de la modularidad
    #Q/Qmax=(S1-S2)/(2m-S2)

    #S1=Suma en nodos (Aij*Cij)
    #S2=Suma en nodos(kikj*Cij/2m)
    S1 = 0
    S2 = 0
    for ni, inodo in enumerate(proteinas):
        for nj, jnodo in enumerate(proteinas):  
            S1 = S1 + A[ni,nj]*C[ni,nj]
            S2 = S2 + grado[inodo]*grado[jnodo]*C[ni,nj]
    S2 = S2/(2*m)
    Q[i] = (S1-S2)/(2*m-S2)
    
Q
    
#  Si da un valor positivo entre 0 y 1, hay más enlaces entre nodos con igual esencialidad que los esperados por azar

array([ 0.07216729,  0.3463515 ,  0.3093429 ,  0.20568567])